In [6]:
from azureml.core import Workspace, Experiment
Workspace.from_config()

Workspace.create(name='quick-starts-ws-122588', subscription_id='a1ebbd25-46bc-446b-9547-5acab9b0125a', resource_group='aml-quickstarts-122588')

In [7]:


ws = Workspace.get(name="quick-starts-ws-122588")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122588
Azure region: southcentralus
Subscription id: a1ebbd25-46bc-446b-9547-5acab9b0125a
Resource group: aml-quickstarts-122588


In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "myNano-proj1"
vm_size = "Standard_D2_V2"
#Verify if the cluster exists
try:
    compute_target = ComputeTarget(workspace=ws,name = cpu_cluster_name)
    print ("Found Existing Cluster")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size= vm_size,max_node=6)
    compute_target = ComputeTarget.create(ws,cpu_cluster_name,compute_config)
compute_target.wait_for_completion(show_output=True)

Found Existing Cluster

Running


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({'learning_rate':uniform(0.000001, 0.1),'keep_probability':uniform(0.05,0.1)})
                                  ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=10)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training',
              entry_script="train.py",
              compute_target=compute_target)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name = "Accuracy_test",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 100,
                                     max_concurrent_runs = 10)### YOUR CODE HERE ###

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

In [20]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_model_hd = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value=best_model_hd,filename='bankmarketing_model.pkl')

['bankmarketing_model.pkl']

In [23]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(data_path)

In [27]:
from train import clean_data

NameError: name 'x' is not defined

In [28]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

NameError: name 'x' is not defined

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='accuracy',
    training_data=x_train,
    label_column_name=label,
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = experiment.submit(automl_config, show_output = False)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_automl, fitted_model_automl = remote_run.get_output()